In [8]:
import pandas as pd
DF = pd.read_csv('train.csv')
test_DF = pd.read_csv('test.csv')
DF.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Preprocessing
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler

# Splitting
from sklearn.model_selection import train_test_split

# Models
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier

# Metrics
from sklearn.metrics import accuracy_score, f1_score

# Grid Search
from sklearn.model_selection import GridSearchCV

In [3]:
DF.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

In [4]:
DF.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [5]:
DF.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [13]:
PASSENGER_ID = test_DF[['PassengerId']]
DF.drop(['PassengerId', 'Cabin', 'Name'], axis=1, inplace=True)
test_DF.drop(['PassengerId', 'Cabin', 'Name'], axis=1, inplace=True)

In [24]:

labels = test_DF.columns
for col in labels:
    if col in ['Age', 'RoomService','FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']:
        DF[col].fillna(DF[col].median(), inplace=True)
        test_DF[col].fillna(DF[col].median(), inplace=True)
    else:
        DF[col].fillna(DF[col].mode()[0], inplace=True)
        test_DF[col].fillna(DF[col].mode()[0], inplace=True)
for col in labels:
    if DF[col].dtype == 'O':
        encoder = LabelEncoder()
        DF[col] = encoder.fit_transform(DF[col])
        test_DF[col] = encoder.transform(test_DF[col])
        
    elif DF[col].dtype == 'bool':
        DF[col] = DF[col].astype('int')
        test_DF[col] = test_DF[col].astype('int')
encoder = LabelEncoder()
DF['Transported'] = DF['Transported'].astype('int')
labels_MM = ['Age']
labels_SS = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

In [25]:
mm_scaler = MinMaxScaler()
ss_scaler = StandardScaler()

DF[labels_MM] = mm_scaler.fit_transform(DF[labels_MM])
test_DF[labels_MM] = mm_scaler.transform(test_DF[labels_MM])


DF[labels_SS] = ss_scaler.fit_transform(DF[labels_SS])
test_DF[labels_SS] = ss_scaler.transform(test_DF[labels_SS])



In [26]:
X, y =DF.drop('Transported', axis=1), DF[['Transported']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, stratify=y)



In [27]:
model_accuracy = {}

In [30]:
ensemble_models = GaussianNB()
predictor = ensemble_models.fit(X_train, y_train)
y_pred = predictor.predict(X_test)
accuracy_gnb = accuracy_score(y_test, y_pred)
model_accuracy['naive_bayes'] = accuracy_gnb
print("Accuracy of Gaussian Naive Bayes :",accuracy_gnb)

Accuracy of Gaussian Naive Bayes : 0.7137931034482758


In [35]:
ensemble_models = LogisticRegression()
predictor = ensemble_models.fit(X_train, y_train)
y_pred = predictor.predict(X_test)
accuracy_lr = accuracy_score(y_test, y_pred)
model_accuracy['naive_bayes'] = accuracy_lr
print("Accuracy of Logistic Regression :",accuracy_lr)

Accuracy of Logistic Regression : 0.7885057471264367


In [36]:
ensemble_models = SVC()
predictor = ensemble_models.fit(X_train, y_train)
y_pred = predictor.predict(X_test)
accuracy_svc = accuracy_score(y_test, y_pred)
model_accuracy['naive_bayes'] = accuracy_svc
print("Accuracy of SVC :",accuracy_svc)

Accuracy of SVC : 0.8080459770114943


In [37]:
ensemble_models = DecisionTreeClassifier()
predictor = ensemble_models.fit(X_train, y_train)
y_pred = predictor.predict(X_test)
accuracy_dtc = accuracy_score(y_test, y_pred)
model_accuracy['naive_bayes'] = accuracy_dtc
print("Accuracy of Decision Tree :",accuracy_dtc)


Accuracy of Decision Tree : 0.7218390804597701


In [38]:
ensemble_models = RandomForestClassifier()
predictor = ensemble_models.fit(X_train, y_train)
y_pred = predictor.predict(X_test)
accuracy_rtc = accuracy_score(y_test, y_pred)
model_accuracy['naive_bayes'] = accuracy_rtc
print("Accuracy of Random Tree :",accuracy_rtc)

Accuracy of Random Tree : 0.7850574712643679


In [39]:
ensemble_models = GradientBoostingClassifier()
predictor = ensemble_models.fit(X_train, y_train)
y_pred = predictor.predict(X_test)
accuracy_gb = accuracy_score(y_test, y_pred)
model_accuracy['naive_bayes'] = accuracy_gb
print("Accuracy of Gradient Boost :",accuracy_gb)

Accuracy of Gaussian Naive Bayes : 0.8057471264367816


In [ ]:
ensemble_models = MLPClassifier()
predictor = ensemble_models.fit(X_train, y_train)
y_pred = predictor.predict(X_test)
accuracy_mlp = accuracy_score(y_test, y_pred)
model_accuracy['naive_bayes'] = accuracy_mlp
print("Accuracy of Gaussian Naive Bayes :",accuracy_mlp)